##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Converting text ops to TF Lite

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/guide/tflite"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/guide/tflite.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/guide/tflite.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/guide/tflite.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

There has been an increased emphasis on serving models on mobile devices with advances in mobile technology and the need for local models to protect privacy and decrease latency. We've taken many of our TensorFlow ops and provide them for use within the TensorFlow Lite environment as well. Performance and size has been the focus for these TF Lite compatible ops. As such, many of these ops have been rewritten from the ground up.

The TensorFlow Lite compatible ops can be grouped into two categories. The first are normal TF Text ops that can be used from within TensorFlow or TF Lite. The second are ops written particularly for the mobile platform in mind and reside in the mobile subpackage of TF Text. Often, these have a modified API from the TensorFlow version with a more limited feature-set, or a fused combination of a common op sequence.

In [1]:
!pip install -q -U tf-nightly
!pip install -q -U tensorflow-text-nightly

## Ops

The following table lists the current ops provided by the TF Text package that can be used from within a TF Lite model. Also listed is the registration function required for inference.

| Op | TF Lite Registration Function |
| --- | --- |
| FastWordpieceTokenizer.detokenize | AddFastWordpieceDetokenize |
| FastWordpieceTokenizer.tokenize | AddFastWordpieceTokenize |
| FastWordpieceTokenizer.tokenizeWithOffsets | AddFastWordpieceTokenize |
| WhitespaceTokenizer.tokenize | AddWhitespaceTokenize |
| WhitespaceTokenizer.tokenizeWithOffsets | AddWhitespaceTokenize |


## Model Example

In [2]:
from absl import app
import numpy as np
import tensorflow as tf
import tensorflow_text as tf_text

from tensorflow.lite.python import interpreter

To show the conversion process and interpretation in Python, we create a very simple test model using the WhitespaceTokenizer. It should be noted that the output of a model cannot be RaggedTensor for TF Lite; however, you can return the components of the RaggedTensor. View this link to learn more about RaggedTensors.

In [3]:
class TokenizerModel(tf.keras.Model):

  def __init__(self, **kwargs):
    super().__init__(**kwargs)
    self.tokenizer = tf_text.WhitespaceTokenizer()

  def call(self, input_tensor, **kwargs):
    return self.tokenizer.tokenize(input_tensor).flat_values

In [4]:
# Test input data.
input_data = np.array([['Some minds are better kept apart']])

# Define a Keras model.
model = TokenizerModel()

# Perform TF.Text inference.
tf_result = model(tf.constant(input_data))
print('TensorFlow result = ', tf_result)


## Convert the TF model to TF Lite

Please review the [TensorFlow Lite converter](https://www.tensorflow.org/lite/convert) documentation for a detailed guide for converting a model. When converting a TensorFlow model with TF Text ops to TF Lite, you need to
indicate to the `TFLiteConverter` that there are custom ops using the
`allow_custom_ops` attribute as in the example below. You can then convert as normal.

In [5]:
# Convert to TFLite.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
converter.allow_custom_ops = True
tflite_model = converter.convert()

## Inference

When reading a model by the TF Lite interpreter, we must indicate to it that we have custom ops, and must provide a registration method for those custom ops. The registration methods are listed above in the [Ops](#scrollTo=_mdIyFfqTMjc) section. Using the `tflite_registrar`, we provide the registration op for the `WhitespaceTokenizer` to the `InterpreterWithCustomOps`.

In [6]:
# Do TFLite inference.
registration_fn = tf_text.tflite_registrar.AddWhitespaceTokenize
interp = interpreter.InterpreterWithCustomOps(
    model_content=tflite_model, custom_op_registerers=[registration_fn])
input_details = interp.get_input_details()

The rest of the steps are common for TF Lite inference. TF Lite must know the input shape beforehand, so we set the input size before calling `allocate_tensors`. Next, we provide the input and then can invoke the interpreter.

In [7]:
interp.resize_tensor_input(input_details[0]['index'], tf.shape(input_data))
interp.allocate_tensors()
interp.set_tensor(input_details[0]['index'], input_data)
interp.invoke()

Finally, we can see our TF Lite result is the same as the TensorFlow result from above.

In [8]:
output_details = interp.get_output_details()
tflite_result = interp.get_tensor(output_details[0]['index'])
print('TFLite result = ', tflite_result)